# Este codigo realiza un proceso de Extracción, Transformación y Carga (ETL) para migrar datos desde la base de datos operacional a la base de datos con modelo de estrella que se usa para generar la analitica del negocio en PowerBI
# 
# **Librerías necesarias:**
# ```bash
# pip install pandas mysql-connector-python sqlalchemy
# ```

In [29]:
# Importaciones y Configuración de la Base de Datos
import pandas as pd
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine

db_source_config = {
    'user': 'root',
    'password': '1qazxsw23edc',
    'host': 'localhost',
    'database': 'tobiolife'
}

db_dest_config = {
    'user': 'root',
    'password': '1qazxsw23edc',
    'host': 'localhost',
    'database': 'tobiolife_analytics'
}

In [30]:
# Conexión a la Base de Datos
def create_db_connection(config):
    """Crea y devuelve una conexión a la base de datos."""
    connection = None
    try:
        connection = mysql.connector.connect(**config)
        print(f"Conexión a la base de datos '{config['database']}' exitosa.")
    except Error as e:
        print(f"Error al conectar a la base de datos '{config['database']}': {e}")
    return connection

def create_sqlalchemy_engine(config):
    """Crea un motor de SQLAlchemy para una carga de datos más eficiente con pandas."""
    try:
        engine_str = f"mysql+mysqlconnector://{config['user']}:{config['password']}@{config['host']}/{config['database']}"
        engine = create_engine(engine_str)
        print(f"Motor de SQLAlchemy para '{config['database']}' creado exitosamente.")
        return engine
    except Exception as e:
        print(f"Error al crear el motor de SQLAlchemy: {e}")
        return None

In [31]:
# Extracción (E) carga de las tablas del modelo relacional en dataframe
def extract_from_source(connection):
    """Extrae todas las tablas necesarias de la base de datos relacional de origen."""
    if not connection:
        return None
        
    tables = [
        "clientes", "aromas", "categoria_opcion", "categoria_producto",
        "pedidos", "catalogo", "productos", "ordenes", "detalle_ordenes"
    ]
    dataframes = {}
    print("\nIniciando extracción de datos desde la base de datos relacional")
    try:
        for table in tables:
            query = f"SELECT * FROM {table}"
            dataframes[table] = pd.read_sql(query, connection)
            print(f"  - Tabla '{table}' extraída exitosamente ({len(dataframes[table])} filas).")
    except Error as e:
        print(f"Error durante la extracción de datos: {e}")
        return None
    
    print("Extracción de datos completada.\n")
    return dataframes

In [32]:
# Transformación (T)
def transform_data(dataframes):
    """Transforma los DataFrames del modelo relacional para que coincidan con el esquema en estrella."""
    if not dataframes:
        return None
        
    print("Iniciando transformación de datos para el modelo en estrella")
    
    # Crear las tablas de dimensiones para el modelo estrella eliminando las claves foráneas y campos innecesarios
    df_dest_clientes = dataframes['clientes'].copy()
    df_dest_aromas = dataframes['aromas'].copy()
    df_dest_categoria_opcion = dataframes['categoria_opcion'].copy()
    df_dest_categoria_producto = dataframes['categoria_producto'].copy()
    
    df_dest_pedidos = dataframes['pedidos'].drop(columns=['cli_id'])
    df_dest_catalogo = dataframes['catalogo'].drop(columns=['co_id'])
    df_dest_productos = dataframes['productos'].drop(columns=['cp_id'])
    
    df_dest_ordenes = dataframes['ordenes'][['ord_id', 'ord_cantidad']].copy()

    # Contruir la tabla de hechos (tabla_hechos) haciendo uniendo las tablas y ordenando los campos

    df_hechos = dataframes['detalle_ordenes'].copy()
    df_hechos = pd.merge(df_hechos, dataframes['ordenes'][['ord_id', 'ped_id', 'opc_id']], on='ord_id', how='left')
    df_hechos = pd.merge(df_hechos, dataframes['pedidos'][['ped_id', 'cli_id']], on='ped_id', how='left')
    df_hechos = pd.merge(df_hechos, dataframes['productos'][['pro_id', 'cp_id']], on='pro_id', how='left')
    df_hechos = pd.merge(df_hechos, dataframes['catalogo'][['opc_id', 'co_id']], on='opc_id', how='left')
    
    column_order = [
        'ped_id', 'cli_id', 'ord_id', 'opc_id', 'co_id', 
        'pro_id', 'cp_id', 'aro_id', 'pro_cantidad'
    ]
    df_dest_tabla_hechos = df_hechos[column_order]
    
    transformed_dfs = {
        'clientes': df_dest_clientes,
        'aromas': df_dest_aromas,
        'categoria_opcion': df_dest_categoria_opcion,
        'categoria_producto': df_dest_categoria_producto,
        'pedidos': df_dest_pedidos,
        'catalogo': df_dest_catalogo,
        'productos': df_dest_productos,
        'ordenes': df_dest_ordenes,
        'tabla_hechos': df_dest_tabla_hechos
    }
    
    print("Transformación de datos completada.\n")
    return transformed_dfs

In [33]:
# Carga (L) de datos en la base de datos analitica
def load_to_destination(transformed_dfs, engine):
    """Carga los DataFrames transformados en la base de datos de destino (modelo estrella)."""
    if not transformed_dfs or not engine:
        return

    print("--- Iniciando carga de datos a la base de datos de destino (estrella) ---")
    
    # Se cargan primero las dimensiones y luego la tabla de hechos con las llaves foraneas
    load_order = [
        'clientes', 'aromas', 'categoria_opcion', 'categoria_producto',
        'pedidos', 'catalogo', 'productos', 'ordenes',
        'tabla_hechos'
    ]
    
    try:
        with engine.connect() as connection:
            for table_name in load_order:
                if table_name in transformed_dfs:
                    df = transformed_dfs[table_name]
                    df.to_sql(table_name, con=connection, if_exists='append', index=False)
                    print(f"Datos cargados exitosamente en la tabla '{table_name}'.")
                else:
                    print(f"No se encontró el DataFrame para la tabla '{table_name}'.")

    except Exception as e:
        print(f"Error durante la carga de datos: {e}")
        return
        
    print("Carga de datos completada.")

In [35]:
# Orquestación del Proceso ETL
def main():
    """Función principal para orquestar el proceso ETL inverso."""
    
    # Conexiones
    source_conn = create_db_connection(db_source_config)
    dest_engine = create_sqlalchemy_engine(db_dest_config)

    if not source_conn or not dest_engine:
        print("No se pudieron establecer las conexiones a la base de datos.")
        return

    # Proceso ETL
    try:
        source_dataframes = extract_from_source(source_conn)

        transformed_dataframes = transform_data(source_dataframes)
        
        load_to_destination(transformed_dataframes, dest_engine)
        
        print("\n¡Proceso ETL completado exitosamente!")
        
    except Exception as e:
        print(f"\nOcurrió un error inesperado durante el proceso ETL: {e}")
        
    finally:
        # Cerrar conexiones
        if source_conn and source_conn.is_connected():
            source_conn.close()
            print("\nConexión a la base de datos de origen cerrada.")
        if dest_engine:
            dest_engine.dispose()
            print("Conexión del motor de destino cerrada.")

if __name__ == "__main__":
    main()

Conexión a la base de datos 'tobiolife' exitosa.
Motor de SQLAlchemy para 'tobiolife_analytics' creado exitosamente.

Iniciando extracción de datos desde la base de datos relacional
  - Tabla 'clientes' extraída exitosamente (17 filas).
  - Tabla 'aromas' extraída exitosamente (11 filas).
  - Tabla 'categoria_opcion' extraída exitosamente (4 filas).
  - Tabla 'categoria_producto' extraída exitosamente (4 filas).
  - Tabla 'pedidos' extraída exitosamente (34 filas).
  - Tabla 'catalogo' extraída exitosamente (29 filas).
  - Tabla 'productos' extraída exitosamente (15 filas).
  - Tabla 'ordenes' extraída exitosamente (59 filas).
  - Tabla 'detalle_ordenes' extraída exitosamente (119 filas).
Extracción de datos completada.

Iniciando transformación de datos para el modelo en estrella
Transformación de datos completada.

--- Iniciando carga de datos a la base de datos de destino (estrella) ---
Datos cargados exitosamente en la tabla 'clientes'.
Datos cargados exitosamente en la tabla 'arom

/var/folders/s8/jqt3gw6j7y99_9h_m8_qww_00000gn/T/ipykernel_39236/2920402036.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table] = pd.read_sql(query, connection)
